In [2]:
import pandas as pd 
import numpy as np
import joblib
import os 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MultiLabelBinarizer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import issparse

                                 Modèle KNN avec des paramètres 

In [18]:
# ---------------------- Custom Transformer ----------------------
class MultiLabelBinarizerPipelineFriendly(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        self.mlb.fit(X)
        return self

    def transform(self, X):
        return self.mlb.transform(X)

    def get_feature_names_out(self, input_features=None):
        return [f"genres_{cls}" for cls in self.mlb.classes_]

# ---------------------- Prétraitement Optimisé ----------------------
preprocessor = ColumnTransformer(
    transformers=[
        ('acteurs', OneHotEncoder(), ['acteurs_1', 'acteurs_2', 'actrices']),
        ('realisateur', OneHotEncoder(), ['realisateurs']),
        ('status', OneHotEncoder(), ['status']),
        ('genres', MultiLabelBinarizerPipelineFriendly(), 'genres'),
        ('annee', OrdinalEncoder(), ['decennie']),
        ('note', 'passthrough', ['vote_average']),
        ('popularite', StandardScaler(), ['popularity']),
        ('duree', StandardScaler(), ['runtimeMinutes'])
    ]
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# ---------------------- Chargement des données ----------------------
films = pd.read_csv("streamlit_app/datasets/raw/films.csv")

# ---------------------- Traitement et compression ----------------------
try:
    df_processed = joblib.load('streamlit_app/datasets/raw/processed_films.pkl')
    nn_model = joblib.load('streamlit_app/datasets/raw/nn_model.pkl')
    distances_all, indices_all = joblib.load('streamlit_app/datasets/raw/nn_distances.pkl')
except FileNotFoundError:
    processed_data = pipeline.fit_transform(films)

    # Si le résultat est sparse, on le convertit en dense
    if issparse(processed_data):
        processed_data = processed_data.toarray()

    # Créer le DataFrame avec colonnes nommées
    columns = pipeline.named_steps['preprocessor'].get_feature_names_out()
    df_processed = pd.DataFrame(processed_data, columns=columns)

    # Optimisation des types numériques
    for col in df_processed.select_dtypes(include='float64'):
        df_processed[col] = df_processed[col].astype('float32')
    for col in df_processed.select_dtypes(include='int64'):
        df_processed[col] = df_processed[col].astype('int32')

    # Sauvegarde compressée
    joblib.dump(df_processed, 'streamlit_app/datasets/raw/processed_films.pkl', compress=3)

    nn_model = NearestNeighbors(n_neighbors=10, algorithm='auto', metric='euclidean')
    nn_model.fit(df_processed)
    joblib.dump(nn_model, 'streamlit_app/datasets/raw/nn_model.pkl', compress=3)

    distances_all, indices_all = nn_model.kneighbors(df_processed)
    joblib.dump((distances_all, indices_all), 'streamlit_app/datasets/raw/nn_distances.pkl', compress=3)

# ---------------------- Vérifier les tailles ----------------------
for path in ['streamlit_app/datasets/raw/processed_films.pkl',
             'streamlit_app/datasets/raw/nn_model.pkl',
             'streamlit_app/datasets/raw/nn_distances.pkl']:
    size = os.path.getsize(path) / 1024 / 1024
    print(f"{path} → {size:.2f} Mo")

# ---------------------- Fonction de recommandation ----------------------
def recommend_similar_items(index: int,
                             data: pd.DataFrame = df_processed,
                             model: NearestNeighbors = nn_model,
                             original_data: pd.DataFrame = films,
                             distances=distances_all,
                             indices=indices_all,
                             n_neighbors: int = 5,
                             sort_by: str = None,
                             min_popularity: float = None,
                             runtime_range: tuple = None) -> pd.DataFrame:

    neighbor_distances = distances[index][0:n_neighbors+1]
    neighbors_idx = indices[index][0:n_neighbors+1]

    recommendations = original_data.iloc[neighbors_idx].copy()
    recommendations['distance'] = neighbor_distances

    if min_popularity is not None:
        recommendations = recommendations[recommendations['popularity'] >= min_popularity]

    if runtime_range is not None:
        min_runtime, max_runtime = runtime_range
        recommendations = recommendations[
            recommendations['runtimeMinutes'].between(min_runtime, max_runtime)
        ]

    if sort_by and sort_by in recommendations.columns:
        recommendations = recommendations.sort_values(by=sort_by, ascending=False)

    print(f"\nRecommandations pour le film : {original_data.iloc[index]['original_title']}\n")
    return recommendations

# ---------------------- Exemple d'utilisation ----------------------
display(recommend_similar_items(
    index=559,
    n_neighbors=10
))


streamlit_app/datasets/raw/processed_films.pkl → 6.14 Mo
streamlit_app/datasets/raw/nn_model.pkl → 5.21 Mo
streamlit_app/datasets/raw/nn_distances.pkl → 0.61 Mo

Recommandations pour le film : All About the Benjamins



,backdrop_path,id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,...,acteurs_1,acteurs_2,actrices,realisateurs,trailer_url,actors_1_posters,actors_2_posters,actress_1_posters,directors_1_posters,distance
559,/3Z9n1DjfYzvknQvoLLkMCvQsSxz.jpg,13950,en,All About the Benjamins,Bucum Jackson (Cube) is a bounty hunter with a...,10.359,/sO7IpJHjZBGLevi8Ozj9EsAUw5h.jpg,['US'],2002-03-08,"['en', 'fr']",...,Ice Cube,Mike Epps,Eva Mendes,Kevin Bray,https://www.youtube.com/watch?v=9Y1DkQr2EHU,/ymR7Yll7HjL6i6Z3pt435hYi91T.jpg,/kDI4E3ALm2rVkF38GwhWwXmDq0a.jpg,/uj6IaUDEUB7swj46dPjHAi3TEk4.jpg,/aQcedywDVuXZhjWoC8ymkjkGauE.jpg,0.000000
451,/sfSXkCTeJ7qBJm44V65u3y7YQf6.jpg,11847,cn,特務迷城,A fun-filled story about an ordinary guy about...,6.262,/fPW4Qn9wm82uOexHMB3TC8x6lRU.jpg,"['HK', 'TR']",2001-01-18,"['ko', 'en', 'fr', 'tr', 'zh', 'cn']",...,Jackie Chan,Eric Tsang,徐若瑄,陳德森,https://www.youtube.com/watch?v=FPdVCH1mIiw,/nraZoTzwJQPHspAVsKfgl3RXKKa.jpg,/vrPBadugqlMkfNIYzLZbP4sEF1j.jpg,/l5uEFLYBSpZJH1I0VkVFfIeI299.jpg,/dSILKNd1loo4j0JSUBXbX0xwdLO.jpg,3.199624
2818,/53GArASULjCKvEXcdQl3bwG8E5E.jpg,433623,km,Jailbreak,What started as a simple escort mission will s...,8.498,/tlNucd0Yi7cIN6DFsa2SoJw5cMm.jpg,['KH'],2017-01-31,"['fr', 'en', 'km']",...,Jean-Paul Ly,Savin Phillip,Céline Tran,Jimmy Henderson,https://www.youtube.com/watch?v=vtADEKSAvvY,/eyryGBcfY0b6Se2EqmNVpbPmBtR.jpg,/wL6VvpT67w3FtT1bvnjTeLOSIxb.jpg,/A0cmXLkVrVTL6JCTuxNZPYwIh60.jpg,NaN,3.337591
129,/xSnKDHTk79wu2UJqI0RDm4tiBvj.jpg,2332,fr,Taxi 2,A cabdriver and a cop race to Paris to rescue ...,16.790,/z0js1eYtKxfw4RBxiTukO7q66Rf.jpg,['FR'],2000-03-25,"['de', 'fr', 'ja']",...,Samy Naceri,Frédéric Diefenthal,Marion Cotillard,Gérard Krawczyk,https://www.youtube.com/watch?v=l58OT6CSHvc,/yT3QZbpp6Pd6xJ2FzxESQ7sDudm.jpg,/3sdQSCxXQTgPu6fqTImqGnebcuU.jpg,/biitzOF0GffIqFYLyOPkoiaOngQ.jpg,/3As3jvza5WtPIbIbLl8hYQToNmV.jpg,3.365564
314,/gtDuUEprJkTIrzcO2vFsXgdkJ0R.jpg,9486,en,Johnny English,"A lowly pencil pusher working for MI7, Johnny ...",18.092,/lx3Y7teA6DaoQ6cIhwzrlnDZpFW.jpg,['GB'],2003-03-14,"['ja', 'en', 'fr']",...,Rowan Atkinson,Ben Miller,Natalie Imbruglia,Peter Howitt,https://www.youtube.com/watch?v=IDUkzZRpA0E,/wxTgS4SFanVKbnvu9xqOzNJWJwz.jpg,/ij5nhtBWhCaOl4IEn8XhOYAGqB1.jpg,/7y56zQpHkbGQ4QPTv7tdOSpcpe5.jpg,/rULzO7TsxYzdfgevc2kILxsoYWN.jpg,3.384781
165,/y1XQMs635Hkj9W476wJXArKVuvw.jpg,3989,en,Team America: World Police,Team America World Police follows an internati...,11.969,/m1Q2VFe1DVVbjfu1LDZe7tlp9yb.jpg,['US'],2004-10-10,"['ar', 'en', 'fr', 'ko']",...,Trey Parker,Matt Stone,Kristen Miller,Trey Parker,https://www.youtube.com/watch?v=kkUkqKSJaVY,/pVEfyxGOBoKoirRZtmSsJ7PX91V.jpg,/nS3BOaF0EHd7vyATV7WBnFEvTLr.jpg,/plgKgewVheRyAPbWZSGzo9Zksfq.jpg,/pVEfyxGOBoKoirRZtmSsJ7PX91V.jpg,3.409764
3700,/mlSHXb7atpwdeQVfAZPaJ6lQxVL.jpg,37632,en,The Order,"When Rudy, an artifacts smuggler, goes to Jeru...",6.709,/xcs2taqwgKu2tDrcXd2k7XbtmLq.jpg,"['AW', 'US']",2001-05-05,"['ar', 'en', 'fr', 'he', 'ru']",...,Jean-Claude Van Damme,Charlton Heston,Sofia Milos,Sheldon Lettich,https://www.youtube.com/watch?v=8ImEUX53wZk,/7DP3bm3MS0qa0L8tBLHGebbCu6H.jpg,/mJabHWZrjlFiF8U8Bbr8lSFtLX6.jpg,/ryK5xo9FTDAtTkC0P4oDes3szS1.jpg,/eTwzTAJ4X48FalI9eDDsyvpooOP.jpg,3.437600
4007,/g3vyN5x5D2vGD2c7n6WPlzCD2c8.jpg,47423,cn,縱橫四海,Three orphans grow up to become art thieves un...,8.901,/gVH8aR8oy4wBlEzIhFXCWLfts9A.jpg,['HK'],1991-02-02,"['cn', 'fr', 'de']",...,Chow Yun-fat,Leslie Cheung,鍾楚紅,John Woo,https://www.youtube.com/watch?v=yiRPGPqPNNI,/uDPYSiZrW7OZ79qgcgkWefALlXY.jpg,/xdSUevukGCrayOdLd4Kd6YSJnKR.jpg,/wtCGaErQlkMdW2nz8br1FsedsU5.jpg,/s7sDmQq8cegHnsTmD0u2eZ4ArYZ.jpg,3.485566
3894,/82NUL0v3bJupk1szqUUOtiPKMfd.jpg,43616,fr,Le roi de l'évasion,"43 years old Armand Lacourtade, a man who sell...",2.595,/yZ9QTedkbBUR5HkwVVcmulTfJUi.jpg,['FR'],2009-07-15,['fr'],...,Ludovic Berthillot,Luc Palun,Hafsia Herzi,Alain Guiraudie,https://www.youtube.com/watch?v=mT6DjqfG

    Modèle prédisant un ou plus de films ayant au moins un des acteurs/actrices

In [20]:
def recommend_by_actors(index: int,
                        original_data: pd.DataFrame = films,
                        min_popularity: float = None,
                        decade: str = None,
                        sort_by: str = 'popularity',
                        top_n: int = 10) -> pd.DataFrame:
    def normalize_name(name):
        return name.strip().lower().replace("é", "e").replace("-", " ")

    # Nettoyage
    for col in ['acteurs_1', 'acteurs_2', 'actrices']:
        original_data[col] = original_data[col].fillna('').astype(str).str.strip().str.title()

    film_ref = original_data.iloc[index]
    acteurs_ref = set([
        film_ref['acteurs_1'],
        film_ref['acteurs_2'],
        film_ref['actrices']
    ])

    def has_common_actor(row):
        acteurs_row = set([
            row['acteurs_1'],
            row['acteurs_2'],
            row['actrices']
        ])
        return not acteurs_ref.isdisjoint(acteurs_row)

    filtered = original_data[original_data.index != index]
    filtered = filtered[filtered.apply(has_common_actor, axis=1)]

    if min_popularity is not None:
        filtered = filtered[filtered['popularity'] >= min_popularity]

    if decade is not None:
        filtered = filtered[filtered['decennie'] == decade]

    if sort_by and sort_by in filtered.columns:
        filtered = filtered.sort_values(by=sort_by, ascending=False)

    print(f"\nFilms avec {', '.join(acteurs_ref)} comme acteur(s)/actrice(s):\n")
    print(f"\nRecommandations avec au moins un acteur commun avec « {film_ref['original_title']} » :\n")
    return filtered.head(top_n)

display(recommend_by_actors(
    index= 42,
    min_popularity= None,
    top_n=10
))


Films avec Laurence Fishburne, Keanu Reeves, Carrie-Anne Moss comme acteur(s)/actrice(s):


Recommandations avec au moins un acteur commun avec « The Matrix Revolutions » :



,backdrop_path,id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,...,genres,acteurs_1,acteurs_2,actrices,realisateurs,trailer_url,actors_1_posters,actors_2_posters,actress_1_posters,directors_1_posters
1825,/h8gHn0OzBoaefsYseUByqsmEDMY.jpg,603692,en,John Wick: Chapter 4,"With the price on his head ever increasing, Jo...",1170.1780,/vZloFAK7NmvMGKE7VkF5UHaz0I.jpg,"['DE', 'US']",2023-03-22,"['cn', 'fr', 'en', 'es', 'it', 'ja', 'la', 'ru']",...,"['Action', 'Thriller', 'Crime']",Keanu Reeves,Donnie Yen,Rina Sawayama,Chad Stahelski,https://www.youtube.com/watch?v=JjBZ2iEBcxM,/8RZLOyYGsoRe9p44q3xin9QkMHv.jpg,/hTlhrrZMj8hZVvD17j4KyAFWBHc.jpg,/yoo4ZcHjF4G51UDBj7omUQrClYI.jpg,/eRCryGwKDH4XqUlrdkERmeBWPo8.jpg
7489,/5IbxcBXZ5EhVAttfNDMPncikudr.jpg,541671,en,Ballerina,Durant les événements de John Wick : Parabellu...,218.1069,/e7zUVzux574daVsOlbcvmqEieyn.jpg,['United States of America'],2025-06-04,['en'],...,"['Action', 'Thriller', 'Crime']",Keanu Reeves,Norman Reedus,Ana De Armas,Len Wiseman,https://www.youtube.com/watch?v=FuO1fbB81pE,/8RZLOyYGsoRe9p44q3xin9QkMHv.jpg,/ozHPdO5jAt7ozzdZUgyRAMNPSDW.jpg,/3vxvsmYLTf4jnr163SUlBIw51ee.jpg,/aXsTPBb6dQ2T3PxLqy2ijf2nxrG.jpg
98,/6KfQTuw4X3LQN11sDA18eYy9HJY.jpg,1921,en,Sweet November,Nelson is a man devoted to his advertising car...,15.0200,/3biEHZSBSYqSkawU2sQyNbLcjgE.jpg,['US'],2001-02-16,"['en', 'fr']",...,"['Drame', 'Romance']",Keanu Reeves,Jason Isaacs,Charlize Theron,Pat O'Connor,https://www.youtube.com/watch?v=cYR4vL5ydUI,/8RZLOyYGsoRe9p44q3xin9QkMHv.jpg,/s6XRFjqUsrDJfDQuXPOoExAYPmb.jpg,/ie1KbeYFG5E0GVr1QP7tDNuXvga.jpg,/zGk61362umO8TtEOuBbU2trwANs.jpg
236,/xTQXtSQQZY5ZbJ1GxEcSCRP4aWV.jpg,6964,en,Something's Gotta Give,Harry Sanborn is an aged music industry exec w...,13.9300,/4fQJjwbDf1KELvJJMcmAV6AxJdH.jpg,['US'],2003-12-12,"['fr', 'en']",...,"['Drame', 'Comédie', 'Romance']",Jack Nicholson,Keanu Reeves,Diane Keaton,Nancy Meyers,https://www.youtube.com/watch?v=fyB1Awfq3YU,/hBHcQIEa6P48HQAlLZkh0eKSSkG.jpg,/8RZLOyYGsoRe9p44q3xin9QkMHv.jpg,/A8B3BsFgbmw2WEmJuQX38qeU9eR.jpg,/oguBZ1ubDnY1FTrUXxjULulbXKV.jpg
7830,/tSxbUnrnWlR5dQvUgqMI7sACmFD.jpg,779782,en,The School for Good and Evil,L'École du Bien et du Mal a été fondée jadis p...,8.2246,/rnCqC51BLayhvziN7bGN8rJFn6X.jpg,['United States of America'],2022-10-19,"['de', 'en', 'fr']",...,"['Fantastique', 'Comédie', 'Aventure', 'Drame']",Laurence Fishburne,Jamie Flatters,Sofia Wylie,Paul Feig,https://www.youtube.com/watch?v=6zMg5E_F6mE,/2GbXERENPpl5MmlqOLlPVaVtifD.jpg,/iOuYlzkBHpU5VcSyO0gQz0QbJTJ.jpg,/l0fOCI0Mj3Z1GfBCGjAQ7692qFG.jpg,/mE3eFY3gq1UQ7Rj2He2YbZ7PFya.jpg
8466,/dUbOGH7CWEIr2aCZBpG2n33Vhw2.jpg,1141182,en,Die Alone,Perdu dans un monde reconquis par la nature et...,2.9017,/WZx9Pe8SMuCBLUvML7VrNGZzE7.jpg,['Canada'],2024-09-27,['en'],...,"['Horreur', 'Thriller', 'Science-Fiction']",Douglas Smith,Frank Grillo,Carrie-Anne Moss,Lowell Dean,https://www.youtube.com/watch?v=XWEt2FoIqlU,/rHZPWNxjIrpiHQnEOttscjTVvnm.jpg,/br2nPzelch2Tb3pZHnYAbXng7cz.jpg,/gc7JwuLDD0kXHUlGx5vWzdlqSIT.jpg,/cTlTYT0JELVsXmPfXY7AVvFq36B.jpg
8823,/ptwrDEMi22n9ejCA7DIs0igF7f4.jpg,916728,en,Slingshot,L'astronaute John lutte pour conserver sa sant...,2.1461,/tYhgXI3XxgWQGLL3BdnRVHHc5fw.jpg,"['Hungary', 'United States of America']",2024-08-30,['en'],...,"['Science-Fiction', 'Thriller']",Casey Affleck,Laurence Fishburne,Emily Beecham,Mikael Håfström,https://www.youtube.com/watch?v=q5blbNc33m4,/7Sn4vA1l9KHIXDT1PH1luSV25KA.jpg,/2GbXERENPpl5MmlqOLlPVaVtifD.jpg,/knR5avTlO4Xnxw7Nc3xxUj3rtjh.jpg,/7zOeNtAlm2aSz1ku5fcsuBFG46j.jpg
9023,/3CXCdAhvHefOUztcUMboPQJskWP.jpg,1361292,en,Cellar Door,Un couple cherche à prendre un nouveau départ ...,1.8244,/4MlL5dG6TAFK1iDhxb3WIXkpDs3.jpg,['United States of America'],2024-10-31,['en'],...,"['Thriller', 'Horreur']",Scott Speedman,Laurence Fishburne,Jordana Brewster,Vaughn Stein,https://www.youtube.com/watch?v=54ecFoxQtN8,/d0fzWMIzsMAXS2M04SKFMof6zQX.jpg,/2GbXERENPpl5MmlqOLlPVaVtifD.j

                      Modèle montrant les films joués par  l'acteur ou l'actrice

In [21]:
def normalize_name(name):
    return name.strip().lower().replace("é", "e").replace("è", "e").replace("-", " ").replace("_"," ").replace("ê"," ").replace("ô","o")

def get_films_by_actor(actor_name: str, original_data: pd.DataFrame = films) -> pd.DataFrame:
    # Normaliser les colonnes d'acteurs
    for col in ['acteurs_1', 'acteurs_2', 'actrices']:
        original_data[col] = original_data[col].fillna('').astype(str).apply(normalize_name)

    actor_name_normalized = normalize_name(actor_name)

    filtered = original_data[
        (original_data['acteurs_1'] == actor_name_normalized) |
        (original_data['acteurs_2'] == actor_name_normalized) |
        (original_data['actrices'] == actor_name_normalized)
    ]

    print(f"\nFilms avec l'acteur/l'actrice : {actor_name.title()}\n")
    return filtered[['original_title', 'decennie', 'popularity', 'vote_average']]
display(get_films_by_actor("Keanu Reeves"))



Films avec l'acteur/l'actrice : Keanu Reeves



,original_title,decennie,popularity,vote_average
42,The Matrix Revolutions,2000,26.0490,6.705
98,Sweet November,2000,15.0200,6.870
236,Something's Gotta Give,2000,13.9300,6.622
1825,John Wick: Chapter 4,2020,1170.1780,7.926
7489,Ballerina,2020,218.1069,7.100


                 Liste des films d'un directeur ou réalisateur

In [22]:
def recommend_by_director(index: int,
                          original_data: pd.DataFrame = films,
                          min_popularity: float = None,
                          decade: str = None,
                          sort_by: str = 'popularity',
                          top_n: int = 10) -> pd.DataFrame:
    def normalize_name(name):
        return name.strip().lower().replace("é", "e").replace("-", " ")

    # Nettoyage du nom du réalisateur
    original_data['realisateurs'] = original_data['realisateurs'].fillna('').astype(str).apply(normalize_name)

    film_ref = original_data.iloc[index]
    director_ref = normalize_name(film_ref['realisateurs'])

    # Filtrer films avec le même réalisateur (exact match)
    filtered = original_data[(original_data.index != index) & (original_data['realisateurs'] == director_ref)]

    if min_popularity is not None:
        filtered = filtered[filtered['popularity'] >= min_popularity]

    if decade is not None:
        filtered = filtered[filtered['decennie'] == decade]

    if sort_by and sort_by in filtered.columns:
        filtered = filtered.sort_values(by=sort_by, ascending=False)

    print(f"\nFilms réalisés par {director_ref.title()} :\n")
    print(f"\nRecommandations avec le même réalisateur que « {film_ref['original_title']} » :\n")
    return filtered.head(top_n)

display(recommend_by_director(
    index=44,
    sort_by='popularity',
    min_popularity=None,
    top_n=10
))


Films réalisés par Neil Jordan :


Recommandations avec le même réalisateur que « Interview with the Vampire » :



,backdrop_path,id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,...,genres,acteurs_1,acteurs_2,actrices,realisateurs,trailer_url,actors_1_posters,actors_2_posters,actress_1_posters,directors_1_posters
221,/hlkdsnRo2bcLBmjJeP2ZiKqoYeR.jpg,5971,en,We're No Angels,Two escaped cons' only prayer to escape is to ...,8.6850,/5XBVURkON7jyzxr0cONWeTbgAov.jpg,['US'],1989-06-02,"['en', 'fr', 'la']",...,"['Comédie', 'Crime']",robert de niro,sean penn,demi moore,neil jordan,https://www.youtube.com/watch?v=k7_zj-CgdaY,/cT8htcckIuyI1Lqwt1CvD02ynTh.jpg,/9glqNTVpFpdN1nFklKaHPUyCwR6.jpg,/wApParZYBDi4yxekjfxjKEifJYh.jpg,/uZ7CvCLTKb9qxjU4VsTNeyMRucd.jpg
8080,/ipy6YEHhqwsEsLX61i6aidaPNOl.jpg,844417,en,Marlowe,"En 1939, à Bay City en Californie, alors que l...",4.7388,/msjaY1sbOpJCOUfpzvZ7uiht3Cc.jpg,"['France', 'Ireland', 'Spain', 'United States ...",2023-02-15,['en'],...,['Crime'],liam neeson,danny huston,diane kruger,neil jordan,https://www.youtube.com/watch?v=MyEBSJcJ-Uo,/sRLev3wJioBgun3ZoeAUFpkLy0D.jpg,/7Wdh73SltnllH4KVd64PvqKQ5NJ.jpg,/o5gVowcjVw4ThYi0L3883Y4X7kH.jpg,/uZ7CvCLTKb9qxjU4VsTNeyMRucd.jpg


In [5]:
# Affiches
base_url = "https://image.tmdb.org/t/p/w500"

# Fonction pour générer l'URL complète ou une image par défaut si absente
def generate_poster_url(poster_path):
    if pd.isna(poster_path) or poster_path == '':
        return 'https://placehold.co/300x450?text=No+Image'
    else:
        return base_url + str(poster_path)

# Application de la fonction sur la colonne poster_path
films['poster_url'] = films['poster_path'].apply(generate_poster_url)

# Remplacer les genres par leur version francaise
films['genres']= films['genres'].str.replace('Drama', 'Drame').str.replace('Comedy', 'Comédie').str.replace('Adventure', 'Aventure').str.replace('Family', 'Familial').str.replace('History', 'Histoire').str.replace('Horror', 'Horreur').str.replace('Fantasy', 'Fantastique').str.replace('Sci-Fi', 'Science-Fiction')
films

,backdrop_path,id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,...,acteurs_1,acteurs_2,actrices,realisateurs,trailer_url,actors_1_posters,actors_2_posters,actress_1_posters,directors_1_posters,poster_url
0,/lVy5Zqcty2NfemqKYbVJfdg44rK.jpg,24,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",31.5560,/v7TaX8kXMXs5yFFGR41guUDNcnB.jpg,['US'],2003-10-10,"['en', 'ja', 'fr']",...,David Carradine,Michael Madsen,Uma Thurman,Quentin Tarantino,https://www.youtube.com/watch?v=d48qtYoeIqE,/1X2GlkMKS9FIG1kGou7o6LRqAjz.jpg,/2pKJJKeCggtLOE4hzqgq07KYqFh.jpg,/lg04iEqT6TC40H1jz10Z99OFMXx.jpg,/1gjcpAa99FAOWGnrUvHEXXsRs7o.jpg,https://image.tmdb.org/t/p/w500/v7TaX8kXMXs5yF...
1,/ELsTifJ2lu4vsMhoHeZ5EnncHw.jpg,75,en,Mars Attacks!,A fleet of Martian spacecraft surrounds the wo...,21.0510,/hll4O5vSAfnZDb6JbnP06GPtz7b.jpg,['US'],1996-12-12,"['en', 'fr']",...,Jack Nicholson,Pierce Brosnan,Glenn Close,Tim Burton,https://www.youtube.com/watch?v=NzC5qlbSNAs,/hBHcQIEa6P48HQAlLZkh0eKSSkG.jpg,/dzXVwwJLPwiZeXOnf7YxorqVEEM.jpg,/eNTn3OmlnsQYLIp7HVGeH9kOJiA.jpg,/yHEHAHQpN9PfSEQx1UxZPczhcAi.jpg,https://image.tmdb.org/t/p/w500/hll4O5vSAfnZDb...
2,/qA2TyqPldTtoTVY3LKrNIG5g6bH.jpg,76,en,Before Sunrise,A young man and woman meet on a train in Europ...,22.3250,/kf1Jb1c2JAOqjuzA3H4oDM263uB.jpg,"['AT', 'CH', 'US']",1995-01-27,"['en', 'de', 'fr']",...,Ethan Hawke,Hanno Pöschl,Julie Delpy,Richard Linklater,https://www.youtube.com/watch?v=IQ0ZrOYMdtA,/slO6aoxA5Pwiy42yt8Fxz5bm1kH.jpg,/t0ZgFxurrzq5jp7wiy00L7SFanf.jpg,/xDnHw9cgIPxj2IQ5p7kW5FIBMZu.jpg,/xSctxeZQzQbZoaxIvcDeHlDjbiN.jpg,https://image.tmdb.org/t/p/w500/kf1Jb1c2JAOqju...
3,/zwgDZw9dyrgkYz2RCOb2HvUDlt2.jpg,80,en,Before Sunset,"Nine years later, Jesse travels across Europe ...",16.1840,/gycdE1ARByGQcK4fYR2mgpU6OO.jpg,['US'],2004-02-10,"['en', 'fr']",...,Ethan Hawke,Vernon Dobtcheff,Julie Delpy,Richard Linklater,https://www.youtube.com/watch?v=at3hf6am2OQ,/slO6aoxA5Pwiy42yt8Fxz5bm1kH.jpg,/o3zjo0Q8BDJQFqHMIIaFbUnckBY.jpg,/xDnHw9cgIPxj2IQ5p7kW5FIBMZu.jpg,/xSctxeZQzQbZoaxIvcDeHlDjbiN.jpg,https://image.tmdb.org/t/p/w500/gycdE1ARByGQcK...
4,/jRJrQ72VLyEnVsvwfep8Xjlvu8c.jpg,101,en,Léon: The Professional,"Léon, the top hit man in New York, has earned ...",44.7010,/yI6X2cCM5YPJtxMhUd3dPGqDAhw.jpg,"['US', 'FR']",1994-09-14,"['en', 'fr', 'it']",...,Jean Reno,Gary Oldman,Natalie Portman,Luc Besson,https://www.youtube.com/watch?v=vSi5jdziv3I,/mw0EZJYz3kiFq9fNxsML773gotF.jpg,/2v9FVVBUrrkW2m3QOcYkuhq9A6o.jpg,/qJ0ipMSZkIYvsOPh7e12l0AMoXl.jpg,/dXJdczT1NcvcZtnoWVGT0NmG11v.jpg,https://image.tmdb.org/t/p/w500/yI6X2cCM5YPJtx...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10968,NaN,1217602,en,London Calling,"After fleeing the UK from a job gone wrong, a ...",0.6779,,"['South Africa', 'United States of America']",2025-09-18,['en'],...,Josh Duhamel,Aidan Gillen,Daniah De Villiers,Allan Ungar,NaN,/5Rp9aeWdN2S623KCjwfii9MhDK1.jpg,/fWWRgyJEGjOPcx43CaFiUCWMBLo.jpg,/bs4sO5DFotOBBgaE9Cka0AiSl3y.jpg,/kYaklrhmyLtvwwKeLrs8AerBYtP.jpg,https://placehold.co/300x450?text=No+Image
10969,/7ewBir8AIg2b0r4gOceCWUOyKNF.jpg,835504,es,Cinco lobitos,"Jeune maman, Amaia, vit dans une grande ville ...",0.6777,/2VJcaYzD1gdNPpb4DaXDgmQ9VmK.jpg,['Spain'],2022-05-17,"['eu', 'en', 'es']",...,Ramón Barea,Mikel Bustamante,Laia Costa,Alauda Ruiz de Azúa,https://www.youtube.com/watch?v=oolcO5mJbWs,/7KJMpbljybgVMhHuNosECDrzact.jpg,/aTzyqha9XMs2ggMvu2c579OtJ96.jpg,/kSbdpHWLlfzvOfZKJ0dgP0ieVL6.jpg,/oRfaZ5YAAZHP507ER1s7NXS3c9.jpg,https://image.tmdb.org/t/p/w500/2VJcaYzD1gdNPp...
10970,/ub6b7hylfx4X1lTTPTXoSsS4coB.jpg,1236469,en,Night Shoot,Un projet de documentaire universitaire tourne...,0.6773,/sf0D8Y9xQAYFWfcQid4A0CJmwwG.jpg,['United States of America'],2025-07-03,['en'],...,Derek Evans,Darrell Snedeger,unknown_actress,Taylor Katsanis,https://www.youtube.com/watch?v=KFr7fDX-rL4,/dGDPW2UCSrsSBUHZQX2i6FaN4jB.jpg,/ykmrGbShFFtfxxmSiqE0rXnw

In [6]:
films.to_csv('streamlit_app/datasets/raw/films.csv') 